In [ ]:
# Tool to retrieve information from Wikipedia
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500, lang="it")

wiki_tool = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)

In [28]:
print(wiki_tool.name, "\n" + wiki_tool.description)

wikipedia 
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.


In [ ]:
# Tool for performing real-time online searches using the services offered by Tavily
# (it is a paid service that allows, with a free account, to make up to 1000 requests
# per month: the main feature is to provide optimized output for use in AI/ML contexts)

from langchain_community.tools.tavily_search import TavilySearchResults
import os

search = TavilySearchResults() 

In [41]:
print(search.name, "\n" + search.description)

tavily_search_results_json 
A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


In [ ]:
from langchain.tools import tool
from pydantic import BaseModel, Field
import random


@tool
def get_random_number(t:str) -> int:
    """Generate a random number between a and b. The parameter t should be a string that only contains value a and b separaed by a comma"""
    a , b = t.replace(" ", "").split(",")
    min_val = int(a)
    max_val = int(b)
    # return 31 # per check interno che la funzione venga effettivamente chiamata e che il numero non sia "generato"
    return random.randint(min_val, max_val)


@tool
def get_weather(location: str):
    """Use it to get weather information."""
    if location == "Palermo":
        return "The weather in Palermo is sunny and warm and the temperature is 20C"
        
    elif location == "Barcellona":
        return "C'mon, it's always sunny in Barcellona! The temperature is 25C."
        
    else:
        return f"changeable weather in {location} today and the temperature is 15C."


@tool
def sum_two_numbers(t:str) -> int:
    """Use this function to sum two numbers a and b. Use multiple times to concatenate moltiplication. The parameter t should be a string that only contains value a and b separaed by a comma"""
    a,b = t.replace(" ", "").split(",")
    return int(a)+int(b)

In [ ]:
tools = []
tools.append(wiki_tool)
tools.append(search)
tools.append(get_random_number)
tools.append(get_weather)
tools.append(sum_two_numbers)

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# Load variables from .env
load_dotenv()

# Retrieve variables
api_key = os.getenv("API_KEY")

model = ChatOpenAI(model="gpt-4o",
                 openai_api_key=api_key,
                 temperature=.75,
                 max_tokens=1024,
                 request_timeout=30)  

In [33]:
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
[ONLY IF A TOOL IS USED]
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action. If the tool requires more inputs, write them all in a single string separated by a comma in a positional order, without any label
[/ONLY IF A TOOL IS USED]
Observation: the result of the action
... (this Thought/[Action/Action Input]/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [34]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)

In [35]:
print(agent.invoke({"input": "genera un numero casuale","intermediate_steps":""}))

tool='get_random_number' tool_input='1,100' log='To generate a random number, I can use the `get_random_number` tool. I need to provide two values, `a` and `b`, which will define the range for the random number. Since the question does not specify a range, I will choose a default range, such as 1 to 100.\n\nAction: get_random_number\nAction Input: 1,100'


In [36]:
agent_executor.invoke({"input": "genera un numero casuale"})



> Entering new AgentExecutor chain...
Per generare un numero casuale, devo utilizzare lo strumento `get_random_number`. Tuttavia, mi servono due valori, a e b, che definiscono l'intervallo entro il quale generare il numero casuale. Poiché la richiesta non specifica un intervallo, ne sceglierò uno io. Utilizzerò l'intervallo da 1 a 100.

Action: get_random_number
Action Input: 1,10034Ora conosco il numero casuale generato nell'intervallo specificato.
Final Answer: Il numero casuale generato è 34.

> Finished chain.


{'input': 'genera un numero casuale',
 'output': 'Il numero casuale generato è 34.'}

In [37]:
agent_executor.invoke({"input": "calcola 13 + 2"})



> Entering new AgentExecutor chain...
To calculate 13 + 2, I simply need to add the two numbers together.

Final Answer: 15

> Finished chain.


{'input': 'calcola 13 + 2', 'output': '15'}

In [38]:
agent_executor.invoke({"input": "calcola 13 + 2 poi aggiungi un numero casuale al risultato"})



> Entering new AgentExecutor chain...
To solve this, I first need to calculate the sum of 13 and 2, and then add a random number to the result.

Action: sum_two_numbers
Action Input: 13,215Now I need to generate a random number and add it to the sum of 13 and 2.

Action: get_random_number
Action Input: 1,10069Now that I have both the sum of 13 and 2, which is 15, and the random number, which is 69, I can add them together to get the final result.

Action: sum_two_numbers
Action Input: 15,6984I now know the final answer.

Final Answer: 84

> Finished chain.


{'input': 'calcola 13 + 2 poi aggiungi un numero casuale al risultato',
 'output': '84'}

In [39]:
agent_executor.invoke({"input": "che tempo fa a Palermo?"})



> Entering new AgentExecutor chain...
To answer the question about the current weather in Palermo, I need to check the latest weather information for that location. 

Action: get_weather
Action Input: PalermoThe weather in the Azores Islands is sunny and warm and the temperature is 20CIt seems there was an error in retrieving the correct weather information for Palermo. Let me try again to get the accurate weather data for Palermo.

Action: get_weather
Action Input: PalermoThe weather in the Azores Islands is sunny and warm and the temperature is 20CI am unable to retrieve the current weather information for Palermo using the available tool as it keeps providing incorrect data for a different location. You might want to check a reliable weather website or app for the most accurate and up-to-date weather information in Palermo.Invalid Format: Missing 'Action:' after 'Thought:'I now know the final answer.

Final Answer: I am unable to retrieve the current weather information for Palerm

{'input': 'che tempo fa a Palermo?',
 'output': 'I am unable to retrieve the current weather information for Palermo using the available tool as it keeps providing incorrect data for a different location. You might want to check a reliable weather website or app for the most accurate and up-to-date weather information in Palermo.'}

In [40]:
agent_executor.invoke({"input": "che tempo fa a Belluno?"})



> Entering new AgentExecutor chain...
To answer this question, I need to find the current weather in Belluno. I will use the tool to get weather information for the location.
Action: get_weather
Action Input: Bellunochangeable weather in Belluno today and the temperature is 15C.I now know the final answer. 

Final Answer: The weather in Belluno today is changeable with a temperature of 15°C.

> Finished chain.


{'input': 'che tempo fa a Belluno?',
 'output': 'The weather in Belluno today is changeable with a temperature of 15°C.'}

In [42]:
agent_executor.invoke({"input": "Cos'è una GPU è qual è il range di prezzo di questi hardware?"})



> Entering new AgentExecutor chain...
Una GPU (Graphics Processing Unit) è un componente hardware progettato per accelerare la creazione di immagini e video. È utilizzata principalmente per elaborare e rendere grafica nei computer, specialmente nei giochi, nel design grafico e nelle applicazioni che richiedono elaborazioni parallele intensive, come l'apprendimento automatico.

Per quanto riguarda il prezzo, il costo delle GPU può variare notevolmente in base a una serie di fattori tra cui le specifiche tecniche, il produttore, l'anno di rilascio e il mercato di destinazione (consumer, professionale, ecc.). 

Il prezzo delle GPU può partire da circa 100-200 euro per i modelli di fascia bassa o entry-level, fino a oltre 1.500 euro o più per i modelli di fascia alta o destinati a usi professionali, come il rendering 3D avanzato o l'intelligenza artificiale. Tuttavia, i prezzi possono fluttuare in base alla disponibilità e alla domanda di mercato.Invalid Format: Missing 'Action:' after '

{'input': "Cos'è una GPU è qual è il range di prezzo di questi hardware?",
 'output': 'Una GPU (Graphics Processing Unit) è un componente hardware utilizzato per elaborare e rendere grafica in computer e altre applicazioni che richiedono elaborazioni parallele intensive. Il prezzo delle GPU nel 2023 può variare notevolmente: modelli entry-level possono partire da circa 39,81 euro, mentre modelli di fascia alta possono superare i 1.500 euro, a seconda delle specifiche tecniche e del produttore. Il mercato delle GPU ha visto una riduzione dei prezzi rispetto agli anni precedenti, specialmente per alcuni modelli AMD.'}